In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
from collections import  Counter

import re
import string
from tqdm import tqdm
import itertools

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from collections import defaultdict
from collections import  Counter

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding, LSTM, SpatialDropout1D, Bidirectional
from keras.layers import Dense, Input, Flatten, Dropout, Concatenate
from keras.layers import Conv1D, MaxPooling1D
from keras.optimizers import Adam

In [ ]:
fake_news = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
true_news = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')

In [ ]:
print(fake_news.shape, true_news.shape)

In [ ]:
true_news.head()

In [ ]:
fake_news.head()

**Gather both Fake&True news and label them**

In [ ]:
fake_news['target'] = 0
true_news['target'] = 1

In [ ]:
df  = pd.concat([fake_news, true_news])
df

In [ ]:
# Shuffle data
df = df.sample(frac=1).reset_index(drop = True)

In [ ]:
# Use last 10,000 as test data check as if they have equally both labels (not imbalanced)
df[34898:].target.value_counts()

In [ ]:
# Check date pattern 
date_pat = r"^\w+\s\d+,\s\d\d\d\d"
match_date_pat = [1 if re.match(date_pat, i) else 0 for i in df.date]
df['match_date_pat'] = match_date_pat
df

In [ ]:
# dates that not match pattern "August 1, 2015" are Fake news
print(df.loc[df.match_date_pat == 0, ['date', 'target']].target.sum())
df.loc[df.match_date_pat == 0, ['date', 'target']]

In [ ]:
# Check for using short version of month
short_month = r"(^[A-Za-z]{3}\s\d+,\s\d\d\d\d)"
match_short_month = [1 if re.match(short_month, i) else 0 for i in df.date]
df['match_short_month'] = match_short_month
df

In [ ]:
# dates that use short version also are Fake news
(df.loc[(df.match_short_month == 1) & (df.target == 1), ['date', 'target']].date.str[:3] != 'May').sum()

In [ ]:
# combine all date pattern columns to correct_date_pat column
May = [1 if i[:3] == 'May' else 0 for i in df.date] 
df['May'] = May
correct_date_pat = [1 if (row['match_date_pat'] == 1)&(row['match_short_month'] == 0) | 
                   (row['match_date_pat'] == 1)&(row['May'] == 1) else 0 for index,row in df.iterrows()]
df['correct_date_pat'] = correct_date_pat
df = df.drop(['match_date_pat','match_short_month','May'] , axis = 1)
df

In [ ]:
# all wrong date pattern are Fake news
(df.loc[df.correct_date_pat == 0, 'target']).sum()

**EDA**

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
f, ax = plt.subplots(2,1 , figsize = (10,10))
sns.countplot(df.subject, ax = ax[0])
sns.countplot(df.subject, hue = df.target, ax = ax[1])

If subject is not politicsNews and worldnews, they all are Fake news

**Number of words in news**

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2,figsize=(10,5))
true_len = df[df['target'] == 1]['title'].str.len()
ax1.hist(true_len,color='green', bins = 20)
ax1.set_title('True news')
ax1.set_xlim([0,250])
fake_len = df[df['target'] == 0]['title'].str.len()
ax2.hist(fake_len,color='red', bins = 20)
ax2.set_title('Fake news')
ax2.set_xlim([0,250])
fig.suptitle('Characters in title of news')
plt.show()

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2,figsize=(10,5))
true_len = df[df['target'] == 1]['title'].str.split().map(lambda x: len(x))
ax1.hist(true_len,color='green', bins = 10)
ax1.set_title('True news')
ax1.set_xlim([0,50])
fake_len = df[df['target'] == 0]['title'].str.split().map(lambda x: len(x))
ax2.hist(fake_len,color='red', bins = 10)
ax2.set_title('Fake news')
ax2.set_xlim([0,50])
fig.suptitle('Words in title of news')
plt.show()

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2,figsize=(10,5))
true_len = df[df['target'] == 1]['text'].str.split().map(lambda x: len(x))
ax1.hist(true_len,color='green', bins = 20)
ax1.set_title('True news')
ax1.set_xlim([0,25000])
fake_len = df[df['target'] == 0]['text'].str.split().map(lambda x: len(x))
ax2.hist(fake_len,color='red', bins = 20)
ax2.set_title('Fake news')
ax2.set_xlim([0,25000])
fig.suptitle('Characters in text of news')
plt.show()

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2,figsize=(10,5))
true_len = df[df['target'] == 1]['text'].str.len()
ax1.hist(true_len,color='green', bins = 20)
ax1.set_title('True news')
ax1.set_xlim([0,25000])
fake_len = df[df['target'] == 0]['text'].str.len()
ax2.hist(fake_len,color='red', bins = 20)
ax2.set_title('Fake news')
ax2.set_xlim([0,25000])
fig.suptitle('Words in text of news')
plt.show()

In [ ]:
stop = set(stopwords.words('english'))

In [ ]:
def create_corpus(target):
    corpus=[]
    
    for x in df[df['target'] == target]['text'].str.split():
        for i in x:
            corpus.append(i)
    return corpus

In [ ]:
corpus_0 = create_corpus(0)

dic_0 = defaultdict(int)
for word in corpus_0:
    if word in stop:
        dic_0[word] += 1
        
top_0 = sorted(dic_0.items(), key=lambda x:x[1],reverse=True)[:10] 

In [ ]:
x, y = zip(*top_0)
plt.bar(x,y)

In [ ]:
corpus_1 = create_corpus(1)

dic_1 = defaultdict(int)
for word in corpus_0:
    if word in stop:
        dic_0[word] += 1
        
top_1 = sorted(dic_0.items(), key=lambda x:x[1],reverse=True)[:10] 

In [ ]:
x, y = zip(*top_1)
plt.bar(x,y)

In [ ]:
plt.figure(figsize=(10,5))
dic_0 = defaultdict(int)
special = string.punctuation
for i in (corpus_0):
    if i in special:
        dic_0[i]+=1
        
x,y = zip(*dic_0.items())
plt.bar(x,y)

In [ ]:
plt.figure(figsize=(10,5))
dic_1 = defaultdict(int)
special = string.punctuation
for i in (corpus_1):
    if i in special:
        dic_1[i]+=1
        
x,y = zip(*dic_1.items())
plt.bar(x,y)

In [ ]:
counter_0 = Counter(corpus_0)
most_0 = counter_0.most_common()
x=[]
y=[]
for word,count in most_0[:50]:
    if (word not in stop) :
        y.append(word)
        x.append(count)
sns.barplot(x=x,y=y)

In [ ]:
counter_1 = Counter(corpus_1)
most_1 = counter_1.most_common()
x=[]
y=[]
for word,count in most_1[:50]:
    if (word not in stop) :
        y.append(word)
        x.append(count)
sns.barplot(x=x,y=y)

**Cleaning data**

In [ ]:
def clean(text):
    # remove non-word charecters like #,*,% etc
    text = re.sub(r'\W',' ',text)
    text = re.sub(r'_',' ',text)
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    
    # remove links and non-ASCII characters
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub(r'',text)
    
    # convert abbreviations
    #tokens = word_tokenize(text)
    #tokens = [abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word for word in tokens]
    #text = ' '.join(tokens)              
                   
    # remove HTML TAGs
    html=re.compile(r'<.*?>')
    text = html.sub(r'',text)
    
    # remove Emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    # remove punctuations
    table = str.maketrans('','',string.punctuation)
    text = text.translate(table)
                   
    return text.lower()

In [ ]:
df['cleaned_text'] = df['text'].apply(lambda x : clean(x))
df

In [ ]:
df["tokens"] = df["cleaned_text"].apply(lambda x: word_tokenize(x))
df

In [ ]:
def stem(data):
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in data]
    return ' '.join(words)

In [ ]:
def lemma(data):
    lem = WordNetLemmatizer()
    words = [lem.lemmatize(word,pos='v') for word in data]
    return ' '.join(words)

In [ ]:
df['cleaned_text'] = df['tokens'].apply(lambda x : stem(x))
df['cleaned_text'] = df['tokens'].apply(lambda x : lemma(x))
df

In [ ]:
def remove_stop(data):
    sentence = []
    for i in data.split():
        if i not in stop:
            sentence.append(i)
    return " ".join(sentence)

In [ ]:
corpus_0 = df.loc[df['target'] == 0, 'cleaned_text'].apply(lambda x : remove_stop(x))
corpus_1 = df.loc[df['target'] == 1, 'cleaned_text'].apply(lambda x : remove_stop(x))

In [ ]:
def get_top_news_bigrams(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
plt.figure(figsize=(10,5))
top_news0_bigrams = get_top_news_bigrams(corpus_0)[:10]
y, x = map(list,zip(*top_news0_bigrams))
sns.barplot(x = x, y = y)

In [ ]:
plt.figure(figsize=(10,5))
top_news1_bigrams = get_top_news_bigrams(corpus_1)[:10]
y, x = map(list,zip(*top_news1_bigrams))
sns.barplot(x = x, y = y)

**Bag of Words Counts**

In [ ]:
train_len = df.shape[0] - 5000

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df[:train_len]['cleaned_text'].values, df[:train_len]['target'].values, test_size = 0.2, random_state = 3)
print('Shape of train',len(X_train))
print("Shape of Validation ",len(X_val))

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train_counts = vectorizer.transform(X_train)
X_val_counts = vectorizer.transform(X_val)

In [ ]:
input_dim = X_train_counts.shape[1]

In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=input_dim, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train_counts, y_train,
                        epochs = 10,
                        verbose = False,
                        validation_data = (X_val_counts, y_val),
                        batch_size = 10)

In [ ]:
loss, accuracy = model.evaluate(X_train_counts, y_train, verbose = False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_val_counts, y_val, verbose = False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
plot_history(history)

In [ ]:
X_test_counts = vectorizer.transform(df[train_len:]['cleaned_text'])
preds = model.predict(X_test_counts)

In [ ]:
y_preds = [1 if i > 0.5 else 0 for i in preds]

In [ ]:
fig = plt.figure(figsize=(10, 10))
cm = confusion_matrix(df[train_len:]['target'], y_preds)
plt.imshow(cm, interpolation='nearest', cmap = plt.cm.winter)
plt.title('Confusion matrix', fontsize=30)
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks([])
plt.yticks([])
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'), horizontalalignment="center", 
                 color="white" if cm[i, j] < cm.max() / 2. else "black", fontsize=40)
plt.tight_layout()
plt.ylabel('True label', fontsize=30)
plt.xlabel('Predicted label', fontsize=30)
plt.show()

In [ ]:
df[train_len:][['text','target']]

In [ ]:
text = df[train_len:]['text'].tolist()
target = df[train_len:]['target'].tolist()

In [ ]:
predicted_table = pd.concat([pd.DataFrame(text), pd.DataFrame(target), pd.DataFrame(y_preds)], axis = 1, ignore_index = True)
predicted_table.columns = ['text', 'target', 'predicted']
predicted_table

In [ ]:
predicted_table.loc[(predicted_table['target'] == 0) & (predicted_table['predicted'] == 1)]

In [ ]:
predicted_table['text'][23]